## DL Assignment No. 05

## DL Assignment No. 05
5. Implement the Continuous Bag of Words (CBOW) Model. Stages can be:

a. Data preparation

b. Generate training data

c. Train model

d. Output                #Context word given hai..Window size=2......we need to give Center as Output

                                         #The Cat is eating the mouse it caught last night..
                                        Given... Context word = (eating,the,it,caught)
                                         
                                         Finding Probability..Center Word= mouse

In [4]:
import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import gensim                          #Python Library for topic modelling and document indexing
data=open('C:/Users/priti/Desktop/Assignment5/corona.txt','r')
corona_data = [text for text in data if text.count(' ') >= 2]
vectorize = Tokenizer()
vectorize.fit_on_texts(corona_data)
corona_data = vectorize.texts_to_sequences(corona_data)
total_vocab = sum(len(s) for s in corona_data)
word_count = len(vectorize.word_index) + 1
window_size = 2

In [5]:
def cbow_model(data, window_size, total_vocab):
    total_length = window_size*2
    for text in data:
        text_len = len(text)
        for idx, word in enumerate(text):
            context_word = []
            target   = []            
            begin = idx - window_size
            end = idx + window_size + 1
            context_word.append([text[i] for i in range(begin, end) if 0 <= i < text_len and i != idx])
            target.append(word)
            contextual = sequence.pad_sequences(context_word, total_length=total_length)
            final_target = np_utils.to_categorical(target, total_vocab)
            yield(contextual, final_target) 

In [6]:
model = Sequential()
model.add(Embedding(input_dim=total_vocab, output_dim=100, input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,)))
model.add(Dense(total_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
for i in range(10):
    cost = 0
    for x, y in cbow_model(data, window_size, total_vocab):
        cost += model.train_on_batch(contextual, final_target)
    print(i, cost)

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0


In [8]:
dimensions=100
vect_file = open('C:/Users/priti/Desktop/Assignment5/vector.txt' ,'w')
vect_file.write('{} {}\n'.format(total_vocab, dimensions))

8

In [9]:
weights = model.get_weights()[0]
for text, i in vectorize.word_index.items():
    final_vec = ' '.join(map(str, list(weights[i, :])))
    vect_file.write('{} {}\n'.format(text, final_vec))
vect_file.close()

In [10]:
from gensim.test.utils import datapath
cbow_output = gensim.models.KeyedVectors.load_word2vec_format(datapath('vectors.txt'), binary=False)
cbow_output.most_similar(positive=['virus'])

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\priti\\.conda\\envs\\tf\\lib\\site-packages\\gensim\\test\\test_data\\vectors.txt'